In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## WeGo Public Transit README

[WeGo Public Transit](https://www.wegotransit.com/) is a public transit system serving the Greater Nashville and Davidson County area. WeGo provides local and regional bus routes, the WeGo Star train service connecting Lebanon to downtown Nashville, along with several other transit services.

The data for this project can be downloaded from [here](https://drive.google.com/drive/folders/1L8d3xEaPD13BMz_k-3G8XRRLvPIbNRq9?usp=sharing).

Since 2019, WeGo has been using [**Transit Signal Priority (TSP)**](https://www.wegotransit.com/projects/transit-signal-priority/), a technology that helps to manage traffic flow more efficiently. For buses it reduces wait times at traffic signals by holding green lights longer, shortening red lights or in some cases allowing buses to bypass traffic. 

The data that you have been provided was collected for trips on Route 50, Charlotte Pike. TSP has been used on portions of this route, with different periods of being on or off, either conditionally or unconditionally. For these timespans, TSP was used between White Bridge and MCC, including all intervening timepoints, in both directions.
The important dates are as follows:

* February 3rd @ 12 noon: TSP Turned On (Unconditional)

* February 10th @ 12 noon: TSP Schedule-Conditional Priority Begins (Only buses more than 2 minutes late receive priority)

* April 28th @ 12 noon: TSP Turned Off

* May 5th @ 12 noon: TSP Turned On (Unconditional)

* May 12th @ 12 noon: TSP Headway-Conditional TSP Priority Begins (Only gapped buses with actual leading headway more than 120% of scheduled headway receive priority)


The first main variable you will be studying in this project is **adherence**, which compares the actual departure time to the scheduled time and is included in the ADHERENCE column. A negative adherence value means that a bus left a time point late and a positive adherence indicates that the bus left the time point early. Buses with adherence values beyond negative 6 are generally considered late and beyond positive 1 are considered early. However, there is some additional logic where the staff applies waivers to allow early departures. For example, express buses that have already picked up everyone at a park-and-ride lot and are only dropping off passengers may be allowed to leave early.  Early departures are also permitted at the end of a trip (when TRIP_EDGE = 2), since they do not affect upstream passengers. **Note:** When determining whether a bus is early or late, it is advised that you use the 'ADJUSTED_EARLY_COUNT', 'ADJUSTED_LATE_COUNT', and 'ADJUSTED_ONTIME_COUNT' columns in order to account for the adjustments.

The second main variable you'll be looking at is **headway**.  This is the amount of time between a bus and the prior bus at the same stop. In the dataset, the amount of headway scheduled is contained in the SCHEDULED_HDWY column and indicates the difference between the scheduled time for a particular stop and the scheduled time for the previous bus on that same stop.
This dataset contains a column HDWY_DEV, which shows the amount of deviation from the scheduled headway. **Bunching** occurs when there is shorter headway than scheduled, which would appear as a negative HDWY_DEV value. **Gapping** is when there is more headway than scheduled and appears as a positive value in the HDWY_DEV column. Note that you can calculate headway deviation percentage as HDWY_DEV/SCHEDULED_HDWY. The generally accepted range of headway deviation is 50% to 150% of the scheduled headway, so if scheduled headway is 10 minutes, a headway deviation of up to 5 minutes would be acceptable (but not ideal).

How has TSP affected these two metrics? Keep in mind that there are many other factors that could also be contributing, so be sure to take into account things like day of the week, time of day, time of year (school in session or not), or other factors that may also be affecting adherence or headway deviation.

## Exploring

In [5]:
# Retrieve Data
wego = pd.read_csv("../data/Route 50 Timepoint and Headway Data, 1-1-2023 through 5-12-2025.csv")

In [6]:
# Look at Columns
wego.columns

Index(['CALENDAR_ID', 'SERVICE_ABBR', 'ADHERENCE_ID', 'DATE', 'ROUTE_ABBR',
       'BLOCK_ABBR', 'OPERATOR', 'TRIP_ID', 'OVERLOAD_ID',
       'ROUTE_DIRECTION_NAME', 'TIME_POINT_ABBR', 'ROUTE_STOP_SEQUENCE',
       'TRIP_EDGE', 'LATITUDE', 'LONGITUDE', 'SCHEDULED_TIME',
       'ACTUAL_ARRIVAL_TIME', 'ACTUAL_DEPARTURE_TIME', 'ADHERENCE',
       'SCHEDULED_HDWY', 'ACTUAL_HDWY', 'HDWY_DEV', 'ADJUSTED_EARLY_COUNT',
       'ADJUSTED_LATE_COUNT', 'ADJUSTED_ONTIME_COUNT', 'STOP_CANCELLED',
       'PREV_SCHED_STOP_CANCELLED', 'IS_RELIEF', 'BLOCK_STOP_ORDER',
       'DWELL_IN_MINS'],
      dtype='object')

In [7]:
# CALENDAR time Columns
wego[['CALENDAR_ID', 'SERVICE_ABBR', 'DATE']]

# CalendarID: 1{year}{mm}{dd}
# Service Abbr: 1 = Weekday, 2 = Saturday, 3 = Sunday
# Date: presumably datetime timestamp format

,CALENDAR_ID,SERVICE_ABBR,DATE
0,120230101,3,2023-01-01
1,120230101,3,2023-01-01
2,120230101,3,2023-01-01
3,120230101,3,2023-01-01
4,120230101,3,2023-01-01
...,...,...,...
618993,120250512,1,2025-05-12
618994,120250512,1,2025-05-12
618995,120250512,1,2025-05-12
618996,120250512,1,2025-05-12


In [8]:
# Route Identification Columns
wego[['ROUTE_ABBR', 'OPERATOR', 'TRIP_ID', 'ROUTE_DIRECTION_NAME', 'TRIP_EDGE', 'IS_RELIEF']]

# todo
# REMOVE route_abbr
# Scrutinize route_direction_name
# Possibly use is_relief to filter for normal trips

,ROUTE_ABBR,OPERATOR,TRIP_ID,ROUTE_DIRECTION_NAME,TRIP_EDGE,IS_RELIEF
0,50,2355,332422,TO DOWNTOWN,1,0
1,50,2355,332422,TO DOWNTOWN,0,0
2,50,2355,332422,TO DOWNTOWN,0,0
3,50,2355,332422,TO DOWNTOWN,0,0
4,50,2355,332422,TO DOWNTOWN,0,0
...,...,...,...,...,...,...
618993,50,3077,432121,FROM DOWNTOWN,2,0
618994,50,3246,432353,TO DOWNTOWN,1,0
618995,50,3246,432353,TO DOWNTOWN,2,0
618996,50,2448,432387,TO DOWNTOWN,1,0


In [9]:
# Efficiency Metrics
wego[['ADHERENCE_ID', 'TIME_POINT_ABBR', 'SCHEDULED_TIME', 'ACTUAL_ARRIVAL_TIME', 'ACTUAL_DEPARTURE_TIME', 'ADHERENCE', 'SCHEDULED_HDWY', 'ACTUAL_HDWY', 'HDWY_DEV', 'ADJUSTED_EARLY_COUNT', 'ADJUSTED_LATE_COUNT', 'ADJUSTED_ONTIME_COUNT', 'DWELL_IN_MINS']]

#lOTS of stuff here.. but I think all I need is:
# [['TIME_POINT_ABBR', 'ADHERENCE', 'SCHEDULED_HDWY', 'ACTUAL_HDWY', 'HDWY_DEV', 'ADJUSTED_EARLY_COUNT', 'ADJUSTED_LATE_COUNT', 'ADJUSTED_ONTIME_COUNT']]

# The data dictionary describes the adherence_id as a unique identifier for each record but... I don't think that means anything to me as an analyst.
# If it has no relation to the other records than it has no insightful value.

# The 'time' columns that show the actual times that the buses moved are overshadowed by the adherence and headway columns I believe.
# There are also the on time flag columns that tell me if the bus is considered ontime, so any information I can get from these columns I already have.

# I am not so sure about dwell in mins. I think that the columns I'm keeping provide more insightful value than dwell times.

# REMOVE adherence_id column, scheduled/actual time columns, 

,ADHERENCE_ID,TIME_POINT_ABBR,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT,DWELL_IN_MINS
0,93549161,WALM,05:34:00,05:29:52,05:38:00,-4.000000,NaN,NaN,NaN,0,0,1,8.133333
1,93549162,HLWD,05:40:00,05:42:38,05:42:38,-2.633333,NaN,NaN,NaN,0,0,1,0.000000
2,93549163,WHBG,05:47:00,05:48:29,05:48:29,-1.483333,NaN,NaN,NaN,0,0,1,0.000000
3,93549164,CH46,05:50:00,05:50:34,05:50:34,-0.566666,NaN,NaN,NaN,0,0,1,0.000000
4,93549165,28&CHARL,05:54:00,05:53:14,05:55:23,-1.383333,NaN,NaN,NaN,0,0,1,2.150000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
618993,119531049,HEAD,07:40:00,07:33:12,07:37:42,2.300000,NaN,NaN,NaN,1,0,0,4.500000
618994,119531638,HEAD,15:05:00,14:38:33,15:05:50,-0.833333,NaN,NaN,NaN,0,0,1,27.283333
618995,119531639,MCC4_20,15:18:00,15:18:06,15:18:14,-0.233333,NaN,NaN,NaN,0,0,1,0.133333
618996,119531770,MLKS,15:20:00,15:14:10,15:22:20,-2.333333,10.0,10.883333,0.883333,0,0,1,8.166666


In [10]:
# Other Dimensions / Facts
wego[['BLOCK_ABBR', 'OVERLOAD_ID', 'ROUTE_STOP_SEQUENCE', 'LATITUDE', 'LONGITUDE', 'STOP_CANCELLED', 'PREV_SCHED_STOP_CANCELLED', 'BLOCK_STOP_ORDER']]

# The BLOCK_ABBR shows me what section of the route a stop is on... I don't think that's useful. Details are better?
# The ROUTE_STOP_SEQUENCE identifies the stop point... I don't think that's useful.
# In the data dictionary BLOCK_STOP_ORDER is blank. I'll ask about it but seems like she goes.

# The OVERLOAD_ID can show me routes that are abnormal to exclude.
# LATITUDE and LONGITUDE will be useful for geospatial possibilities.
# STOP_CANCELLED and PREV_SCHED_STOP_CANCELLED could help me exclude abnormal circumstances.
# For PREV_SCHED_STOP_CANCELLED: Useful for excluding records where the headway values are extremely high because the bus is just coming off a detour.

,BLOCK_ABBR,OVERLOAD_ID,ROUTE_STOP_SEQUENCE,LATITUDE,LONGITUDE,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,BLOCK_STOP_ORDER
0,5000,0,17,36.132451,-86.907214,0,0.0,2
1,5000,0,16,36.138535,-86.881130,0,0.0,5
2,5000,0,15,36.149997,-86.858261,0,0.0,11
3,5000,0,14,36.152077,-86.842746,0,0.0,13
4,5000,0,13,36.153870,-86.818414,0,0.0,18
...,...,...,...,...,...,...,...,...
618993,8401,0,12,36.158659,-86.804683,0,NaN,23
618994,9302,0,7,36.158659,-86.804683,0,0.0,2
618995,9302,0,4,36.167091,-86.781923,0,NaN,3
618996,9950,0,11,36.161197,-86.800180,0,0.0,2


In [11]:
# Lets take a closer look at the columns I'm unsure of:
# ROUTE_DIRECTION_NAME: TO_DOWNTOWN
(
    wego
    .loc[wego['ROUTE_DIRECTION_NAME'] == 'TO DOWNTOWN']
    .loc[wego['CALENDAR_ID'] == 120230101]
    .loc[wego['TRIP_ID'] == 332482]
    [[
        'CALENDAR_ID', 'SERVICE_ABBR', 'DATE',
        'OPERATOR', 'TRIP_ID', 'ROUTE_DIRECTION_NAME', 
        'TIME_POINT_ABBR', 'TRIP_EDGE', 'SCHEDULED_TIME',
        'ADHERENCE', 'SCHEDULED_HDWY', 'ACTUAL_HDWY', 
        'HDWY_DEV', 'ADJUSTED_EARLY_COUNT',
        'ADJUSTED_LATE_COUNT', 'ADJUSTED_ONTIME_COUNT'
     ]]
)

,CALENDAR_ID,SERVICE_ABBR,DATE,OPERATOR,TRIP_ID,ROUTE_DIRECTION_NAME,TIME_POINT_ABBR,TRIP_EDGE,SCHEDULED_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
144,120230101,3,2023-01-01,2217,332482,TO DOWNTOWN,WALM,1,06:03:00,-0.983333,29.0,25.983333,-3.016667,0,0,1
145,120230101,3,2023-01-01,2217,332482,TO DOWNTOWN,HLWD,0,06:09:00,0.600000,29.0,25.766666,-3.233334,0,0,1
146,120230101,3,2023-01-01,2217,332482,TO DOWNTOWN,WHBG,0,06:17:00,0.983333,30.0,27.533333,-2.466667,0,0,1
147,120230101,3,2023-01-01,2217,332482,TO DOWNTOWN,CH46,0,06:20:00,0.816666,30.0,28.616666,-1.383334,0,0,1
148,120230101,3,2023-01-01,2217,332482,TO DOWNTOWN,28&CHARL,0,06:24:00,0.866666,30.0,27.750000,-2.250000,0,0,1
149,120230101,3,2023-01-01,2217,332482,TO DOWNTOWN,MCC5_1,2,06:35:00,6.366666,NaN,NaN,NaN,0,0,1


In [12]:
# ROUTE_DIRECTION_NAME: FROM_DOWNTOWN
(
    wego
    .loc[wego['ROUTE_DIRECTION_NAME'] == 'FROM DOWNTOWN']
    .loc[wego['CALENDAR_ID'] == 120230101]
    .loc[wego['TRIP_ID'] == 332423]
    [[
        'CALENDAR_ID', 'SERVICE_ABBR', 'DATE',
        'OPERATOR', 'TRIP_ID', 'ROUTE_DIRECTION_NAME', 
        'TIME_POINT_ABBR', 'TRIP_EDGE', 'SCHEDULED_TIME',
        'ADHERENCE', 'SCHEDULED_HDWY', 'ACTUAL_HDWY', 
        'HDWY_DEV', 'ADJUSTED_EARLY_COUNT',
        'ADJUSTED_LATE_COUNT', 'ADJUSTED_ONTIME_COUNT'
     ]]
)

,CALENDAR_ID,SERVICE_ABBR,DATE,OPERATOR,TRIP_ID,ROUTE_DIRECTION_NAME,TIME_POINT_ABBR,TRIP_EDGE,SCHEDULED_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
6,120230101,3,2023-01-01,2355,332423,FROM DOWNTOWN,MCC5_1,1,06:15:00,-5.200000,NaN,NaN,NaN,0,0,1
7,120230101,3,2023-01-01,2355,332423,FROM DOWNTOWN,28&CHARL,0,06:25:00,-1.916666,NaN,NaN,NaN,0,0,1
8,120230101,3,2023-01-01,2355,332423,FROM DOWNTOWN,CH46,0,06:29:00,-2.483333,NaN,NaN,NaN,0,0,1
9,120230101,3,2023-01-01,2355,332423,FROM DOWNTOWN,WHBG,0,06:33:00,-2.200000,NaN,NaN,NaN,0,0,1
10,120230101,3,2023-01-01,2355,332423,FROM DOWNTOWN,HLWD,0,06:40:00,-0.683333,NaN,NaN,NaN,0,0,1
11,120230101,3,2023-01-01,2355,332423,FROM DOWNTOWN,WALM,2,06:47:00,2.983333,NaN,NaN,NaN,0,0,1


In [13]:
# I know that there are TSP systems only from Charlotte up to Whitebridge Rd, 
# so this ROUTE_DIRECTION_NAME column could be useful for determining that
# I don't think I will know until I get to the meat of it.

In [14]:
# DWELL_IN_MINS
wego['DWELL_IN_MINS'].sort_values(ascending=False).head()

# This doesn't look right... but I'm unsure if I can rule this out yet.
# I believe ADHERENCE and HDWY columns are solely what's necessary to determine efficiency, but this may come in to play.
# Either way it needs some cleaning. Scientific numbers are never a good sign.

346870    2.132813e+06
352298    1.124500e+03
530746    5.762667e+02
308935    4.187667e+02
434647    3.795000e+02
Name: DWELL_IN_MINS, dtype: float64

In [15]:
# I'm curious about the connection between TRIP_ID and ROUTE_DIRECTION_NAME
wego[['TRIP_ID', 'ROUTE_DIRECTION_NAME']].groupby(['TRIP_ID', 'ROUTE_DIRECTION_NAME']).size().reset_index().rename(columns={0:'count'})

,TRIP_ID,ROUTE_DIRECTION_NAME,count
0,332380,TO DOWNTOWN,65
1,332381,FROM DOWNTOWN,60
2,332382,TO DOWNTOWN,60
3,332383,FROM DOWNTOWN,60
4,332384,TO DOWNTOWN,60
...,...,...,...
5720,429701,TO DOWNTOWN,58
5721,430504,FROM DOWNTOWN,56
5722,432121,FROM DOWNTOWN,58
5723,432353,TO DOWNTOWN,60


I like this but I don't think it does anything for me really.
Might be useful for double checking things or looking at specific trips.

## Cleaning

In [18]:
wego.columns

Index(['CALENDAR_ID', 'SERVICE_ABBR', 'ADHERENCE_ID', 'DATE', 'ROUTE_ABBR',
       'BLOCK_ABBR', 'OPERATOR', 'TRIP_ID', 'OVERLOAD_ID',
       'ROUTE_DIRECTION_NAME', 'TIME_POINT_ABBR', 'ROUTE_STOP_SEQUENCE',
       'TRIP_EDGE', 'LATITUDE', 'LONGITUDE', 'SCHEDULED_TIME',
       'ACTUAL_ARRIVAL_TIME', 'ACTUAL_DEPARTURE_TIME', 'ADHERENCE',
       'SCHEDULED_HDWY', 'ACTUAL_HDWY', 'HDWY_DEV', 'ADJUSTED_EARLY_COUNT',
       'ADJUSTED_LATE_COUNT', 'ADJUSTED_ONTIME_COUNT', 'STOP_CANCELLED',
       'PREV_SCHED_STOP_CANCELLED', 'IS_RELIEF', 'BLOCK_STOP_ORDER',
       'DWELL_IN_MINS'],
      dtype='object')

#### Combining Date and Time columns

In [20]:
wego['ACTUAL_DEPARTURE_TIME']

0         05:38:00
1         05:42:38
2         05:48:29
3         05:50:34
4         05:55:23
            ...   
618993    07:37:42
618994    15:05:50
618995    15:18:14
618996    15:22:20
618997    15:31:28
Name: ACTUAL_DEPARTURE_TIME, Length: 618998, dtype: object

In [21]:
wego['ACTUAL_DEPARTURE_TIME'][1539]

'1900-01-01 00:02:04'

In [22]:
wego.iloc[1539:1540][['CALENDAR_ID', 'SERVICE_ABBR', 'DATE',
        'OPERATOR', 'TRIP_ID', 'ROUTE_DIRECTION_NAME', 
        'TIME_POINT_ABBR', 'TRIP_EDGE', 'SCHEDULED_TIME', 'ACTUAL_ARRIVAL_TIME', 'ACTUAL_DEPARTURE_TIME',
        'ADHERENCE', 'SCHEDULED_HDWY', 'ACTUAL_HDWY', 
        'HDWY_DEV', 'ADJUSTED_EARLY_COUNT',
        'ADJUSTED_LATE_COUNT', 'ADJUSTED_ONTIME_COUNT']]

,CALENDAR_ID,SERVICE_ABBR,DATE,OPERATOR,TRIP_ID,ROUTE_DIRECTION_NAME,TIME_POINT_ABBR,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
1539,120230103,1,2023-01-03,1623,332573,FROM DOWNTOWN,WHBG,0,1900-01-01 00:01:00,23:59:47,1900-01-01 00:02:04,-1.066666,30.0,26.766666,-3.233334,0,0,1


In [99]:
# Combining Date and Time columns
# wego['comb_timestamp_departure'] = 
pd.to_datetime(wego['DATE'] + ' ' + pd.to_datetime(wego['ACTUAL_DEPARTURE_TIME'], format='%H:%M:%S', exact=False).astype(str))

C:\Users\nunuk\AppData\Local\Temp\ipykernel_9768\527222452.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(wego['DATE'] + ' ' + pd.to_datetime(wego['ACTUAL_DEPARTURE_TIME'], format='%H:%M:%S', exact=False).astype(str))


DateParseError: Unknown datetime string format, unable to parse: 2023-01-03 NaT, at position 1195

In [23]:
wego['ACTUAL_DEPARTURE_TIME'] = pd.to_datetime(wego['ACTUAL_DEPARTURE_TIME'], format="%H:%M:%S", exact=False).dt.time

In [24]:
# REVISION: Andy shared this code he used to narrow it down by date and flag each row based on tsp status.
# EDIT: I had to edit this code to ensure accuracy.
tsp_window = [
    (wego['DATE']>="2025-02-03") & (wego['DATE']<"2025-02-10") & (wego['ACTUAL_DEPARTURE_TIME'].hour> pd.to_datetime('12:00:00', format="%H:%M:%S")) ,
    (wego['DATE']>="2025-02-10") & (wego['DATE']<"2025-04-28") & (wego['ACTUAL_DEPARTURE_TIME'].dt.time>'12:00:00'),
    (wego['DATE']>="2025-04-28") & (wego['DATE']<"2025-05-05") & (wego['ACTUAL_DEPARTURE_TIME'].dt.time>'12:00:00'),
    (wego['DATE']>="2025-05-05") & (wego['DATE']<"2025-05-12") & (wego['ACTUAL_DEPARTURE_TIME'].dt.time>'12:00:00'),
    (wego['DATE']>="2025-05-12") & (wego['ACTUAL_DEPARTURE_TIME'].dt.time>noon.dt.time)
]
phases = [
    "tsp_on_uncond",
    "tsp_on_sched_cond",
    "tsp_off",
    "tsp_on_uncond",
    "tsp_on_hdwy_cond"
]
wego["tsp_status"] = np.select(tsp_window, phases, default="tsp_off")

AttributeError: 'Series' object has no attribute 'hour'

#### Trimming

In [54]:
# Selecting relevant columns to keep
wego_trim = wego[['CALENDAR_ID', 'SERVICE_ABBR', 'DATE',
        'OPERATOR', 'TRIP_ID', 'ROUTE_DIRECTION_NAME', 
        'TIME_POINT_ABBR', 'TRIP_EDGE', 'SCHEDULED_TIME', 'ACTUAL_ARRIVAL_TIME', 'ACTUAL_DEPARTURE_TIME',
        'ADHERENCE', 'SCHEDULED_HDWY', 'ACTUAL_HDWY', 
        'HDWY_DEV', 'ADJUSTED_EARLY_COUNT',
        'ADJUSTED_LATE_COUNT', 'ADJUSTED_ONTIME_COUNT',
        'STOP_CANCELLED', 'PREV_SCHED_STOP_CANCELLED', 
        'IS_RELIEF']]

In [56]:
# Rename Columns
wego_trim = wego_trim.rename(columns={"CALENDAR_ID": "cal_id", 
                            "SERVICE_ABBR": "service_day",
                            "DATE" : "date",
                            "OPERATOR" : "operator_id",
                            "TRIP_ID" : "trip_id",
                            "ROUTE_DIRECTION_NAME" : "route_direction",
                            "TIME_POINT_ABBR" : "time_point",
                            "TRIP_EDGE" : "trip_edge",
                            "SCHEDULED_TIME" : "sched_time",
                            "ADHERENCE" : "adherence",
                            "SCHEDULED_HDWY" : "sched_hdwy",
                            "ACTUAL_HDWY" : "act_hdwy",
                            "HDWY_DEV" : "hdwy_dev",
                            "ADJUSTED_EARLY_COUNT" : "adj_early",
                            "ADJUSTED_LATE_COUNT" : "adj_late",
                            "ADJUSTED_ONTIME_COUNT" : "adj_ontime"})

### Combining the ajd_early, adj_late, and adj_ontime columns into one flag column. (Andy's Idea)

In [59]:
wego_trim.head(1)

,cal_id,service_day,date,operator_id,trip_id,route_direction,time_point,trip_edge,sched_time,ACTUAL_ARRIVAL_TIME,...,adherence,sched_hdwy,act_hdwy,hdwy_dev,adj_early,adj_late,adj_ontime,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF
0,120230101,3,2023-01-01,2355,332422,TO DOWNTOWN,WALM,1,05:34:00,05:29:52,...,-4.0,NaN,NaN,NaN,0,0,1,0,0.0,0


In [61]:
# Writing flag_sort function to sort and debug
row_indexes = []
row_sums = []

def flag_sort(row):
    if (row['adj_early'] + row['adj_late'] + row['adj_ontime']) == 1:
        if row['adj_early'] == 1:
            return 1
        if row['adj_late'] == 1:
            return 2
        if row['adj_ontime'] == 1:
            return 3
    else:
        row_indexes.append(row.name)
        row_sums.append(row['adj_early'] + row['adj_late'] + row['adj_ontime'])
        print('Error at: ' + str(row.name))
        print('      sum adj cols: ' + str(row['adj_early'] + row['adj_late'] + row['adj_ontime']))

In [63]:
# Testing
wego_trim.iloc[50:60].apply(flag_sort, axis=1)

50    2
51    2
52    2
53    3
54    1
55    3
56    3
57    3
58    3
59    3
dtype: int64

In [65]:
# Checking
wego_trim[['adj_early', 'adj_late', 'adj_ontime']].iloc[50:60]

,adj_early,adj_late,adj_ontime
50,0,1,0
51,0,1,0
52,0,1,0
53,0,0,1
54,1,0,0
55,0,0,1
56,0,0,1
57,0,0,1
58,0,0,1
59,0,0,1


In [67]:
# Blast off
wego_trim['timing_flag'] = wego_trim.apply(flag_sort, axis=1)

Error at: 1195
      sum adj cols: 0
Error at: 1196
      sum adj cols: 0
Error at: 1207
      sum adj cols: 0
Error at: 1208
      sum adj cols: 0
Error at: 1209
      sum adj cols: 0
Error at: 1210
      sum adj cols: 0
Error at: 1211
      sum adj cols: 0
Error at: 1212
      sum adj cols: 0
Error at: 1213
      sum adj cols: 0
Error at: 1214
      sum adj cols: 0
Error at: 1220
      sum adj cols: 0
Error at: 1291
      sum adj cols: 0
Error at: 1292
      sum adj cols: 0
Error at: 1293
      sum adj cols: 0
Error at: 1294
      sum adj cols: 0
Error at: 1295
      sum adj cols: 0
Error at: 1296
      sum adj cols: 0
Error at: 1297
      sum adj cols: 0
Error at: 1298
      sum adj cols: 0
Error at: 1299
      sum adj cols: 0
Error at: 1300
      sum adj cols: 0
Error at: 1809
      sum adj cols: 0
Error at: 1810
      sum adj cols: 0
Error at: 1811
      sum adj cols: 0
Error at: 1812
      sum adj cols: 0
Error at: 1914
      sum adj cols: 0
Error at: 1915
      sum adj cols: 0
E

In [68]:
# I bet you NASAs first launch was a failure too
row_indexes[0]

1195

In [69]:
# My hypothesis is that these rows are missing too much information to calculate their punctuality.
wego_trim.iloc[row_indexes[0]:row_indexes[0]+1]

,cal_id,service_day,date,operator_id,trip_id,route_direction,time_point,trip_edge,sched_time,ACTUAL_ARRIVAL_TIME,...,sched_hdwy,act_hdwy,hdwy_dev,adj_early,adj_late,adj_ontime,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,timing_flag
1195,120230103,1,2023-01-03,664,332455,FROM DOWNTOWN,28&CHARL,0,11:40:00,NaN,...,15.0,NaN,NaN,0,0,0,0,0.0,0,NaN


In [70]:
# Hypothesis correct
wego_trim.iloc[row_indexes[0]:row_indexes[0]+1][['sched_time', 'ACTUAL_ARRIVAL_TIME', 'ACTUAL_DEPARTURE_TIME']]

,sched_time,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME
1195,11:40:00,NaN,NaT


In [71]:
# Now the plan is to use the indexes I saved to remove dirty rows.
len(row_indexes)

18218

In [72]:
# Check original size
wego_trim.shape

(618998, 22)

In [73]:
# This is the number we should get
print(wego_trim.shape[0] - len(row_indexes))

600780


In [81]:
# Drop dirty rows
wego_trim = wego_trim.drop(row_indexes)

In [83]:
# Check new size
wego_trim.shape

(600780, 22)

In [85]:
wego_trim.head(1)

,cal_id,service_day,date,operator_id,trip_id,route_direction,time_point,trip_edge,sched_time,ACTUAL_ARRIVAL_TIME,...,sched_hdwy,act_hdwy,hdwy_dev,adj_early,adj_late,adj_ontime,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,timing_flag
0,120230101,3,2023-01-01,2355,332422,TO DOWNTOWN,WALM,1,05:34:00,05:29:52,...,NaN,NaN,NaN,0,0,1,0,0.0,0,3.0


In [87]:
# It's a float why
wego_trim['timing_flag'] = wego_trim['timing_flag'].astype(int)

In [89]:
# Removing the 3 adj_ columns, now all we need is 'timing_flag'
wego_trim = wego_trim.drop(columns=['adj_early', 'adj_late', 'adj_ontime'])

In [91]:
# Squeaky clean
wego_trim.head()

,cal_id,service_day,date,operator_id,trip_id,route_direction,time_point,trip_edge,sched_time,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,adherence,sched_hdwy,act_hdwy,hdwy_dev,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,timing_flag
0,120230101,3,2023-01-01,2355,332422,TO DOWNTOWN,WALM,1,05:34:00,05:29:52,05:38:00,-4.000000,NaN,NaN,NaN,0,0.0,0,3
1,120230101,3,2023-01-01,2355,332422,TO DOWNTOWN,HLWD,0,05:40:00,05:42:38,05:42:38,-2.633333,NaN,NaN,NaN,0,0.0,0,3
2,120230101,3,2023-01-01,2355,332422,TO DOWNTOWN,WHBG,0,05:47:00,05:48:29,05:48:29,-1.483333,NaN,NaN,NaN,0,0.0,0,3
3,120230101,3,2023-01-01,2355,332422,TO DOWNTOWN,CH46,0,05:50:00,05:50:34,05:50:34,-0.566666,NaN,NaN,NaN,0,0.0,0,3
4,120230101,3,2023-01-01,2355,332422,TO DOWNTOWN,28&CHARL,0,05:54:00,05:53:14,05:55:23,-1.383333,NaN,NaN,NaN,0,0.0,0,3


### Filtering out NaNs

In [41]:
# I love that there is so much missing data in the headway department
wego_trim.head()

,cal_id,service_day,date,operator_id,trip_id,route_direction,time_point,trip_edge,sched_time,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,adherence,sched_hdwy,act_hdwy,hdwy_dev,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,tsp_status,timing_flag
0,120230101,3,2023-01-01,2355,332422,TO DOWNTOWN,WALM,1,05:34:00,05:29:52,05:38:00,-4.000000,NaN,NaN,NaN,0,0.0,0,tsp_off,3
1,120230101,3,2023-01-01,2355,332422,TO DOWNTOWN,HLWD,0,05:40:00,05:42:38,05:42:38,-2.633333,NaN,NaN,NaN,0,0.0,0,tsp_off,3
2,120230101,3,2023-01-01,2355,332422,TO DOWNTOWN,WHBG,0,05:47:00,05:48:29,05:48:29,-1.483333,NaN,NaN,NaN,0,0.0,0,tsp_off,3
3,120230101,3,2023-01-01,2355,332422,TO DOWNTOWN,CH46,0,05:50:00,05:50:34,05:50:34,-0.566666,NaN,NaN,NaN,0,0.0,0,tsp_off,3
4,120230101,3,2023-01-01,2355,332422,TO DOWNTOWN,28&CHARL,0,05:54:00,05:53:14,05:55:23,-1.383333,NaN,NaN,NaN,0,0.0,0,tsp_off,3


In [42]:
# Viewing columns to select important metrics
wego_trim.columns

Index(['cal_id', 'service_day', 'date', 'operator_id', 'trip_id',
       'route_direction', 'time_point', 'trip_edge', 'sched_time',
       'ACTUAL_ARRIVAL_TIME', 'ACTUAL_DEPARTURE_TIME', 'adherence',
       'sched_hdwy', 'act_hdwy', 'hdwy_dev', 'STOP_CANCELLED',
       'PREV_SCHED_STOP_CANCELLED', 'IS_RELIEF', 'tsp_status', 'timing_flag'],
      dtype='object')

In [88]:
# I guess this is it?
dirty_rows = wego_trim[wego_trim[['ACTUAL_ARRIVAL_TIME', 'ACTUAL_DEPARTURE_TIME', 'adherence']].isna().any(axis=1)]
dirty_rows

,cal_id,service_day,date,operator_id,trip_id,route_direction,time_point,trip_edge,sched_time,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,adherence,sched_hdwy,act_hdwy,hdwy_dev,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,tsp_status,timing_flag
217617,120231104,2,2023-11-04,2629,361579,FROM DOWNTOWN,MCC5_1,1,09:15:00,NaN,09:29:28,-14.466666,30.0,43.116666,13.116666,1,0.0,0,tsp_on_hdwy_cond,2
244983,120231212,1,2023-12-12,1791,366776,FROM DOWNTOWN,WALM,2,18:35:00,NaN,18:45:32,-10.533333,NaN,NaN,NaN,0,NaN,0,tsp_on_hdwy_cond,2
315006,120240318,1,2024-03-18,2448,374570,TO DOWNTOWN,MLKS,1,15:20:00,NaN,20:56:40,-336.666666,10.0,345.100000,335.100000,0,0.0,0,tsp_on_hdwy_cond,2
338082,120240419,1,2024-04-19,2137,377471,FROM DOWNTOWN,MCC5_1,1,07:00:00,NaN,07:03:18,-3.300000,15.0,0.533333,-14.466667,1,0.0,0,tsp_on_hdwy_cond,3
452155,120240924,1,2024-09-24,664,394633,FROM DOWNTOWN,MCC5_1,1,06:15:00,NaN,06:15:20,-0.333333,35.0,32.733333,-2.266667,0,0.0,0,tsp_on_hdwy_cond,3
506322,120241207,2,2024-12-07,3388,406166,FROM DOWNTOWN,MCC5_1,1,15:15:00,NaN,15:38:10,-23.166666,20.0,38.366666,18.366666,0,0.0,0,tsp_on_hdwy_cond,2
508794,120241211,1,2024-12-11,1457,406025,TO DOWNTOWN,28&CHARL,0,12:24:00,NaN,13:55:14,-91.233333,15.0,12.516666,-2.483334,0,0.0,0,tsp_on_hdwy_cond,2
522509,120241230,1,2024-12-30,3124,412017,FROM DOWNTOWN,MCC5_1,1,20:15:00,NaN,20:19:42,-4.700000,30.0,32.783333,2.783333,0,0.0,0,tsp_on_hdwy_cond,3
548103,120250204,1,2025-02-04,1623,417857,TO DOWNTOWN,WALM,1,19:32:00,NaN,20:17:04,-45.066666,31.0,3.900000,-27.100000,0,0.0,0,tsp_on_hdwy_cond,2
563932,120250226,1,2025-02-26,1893,423678,FROM DOWNTOWN,MCC5_1,1,12:30:00,NaN,15:44:38,-194.633333,15.0,11.033333,-3.966667,0,0.0,0,tsp_on_hdwy_cond,2


In [106]:
# Show the full trips associated with the dirty rows
dirty_df = (
    wego_trim
    .loc[wego_trim['cal_id'].isin(dirty_rows['cal_id'])]
    .loc[wego_trim['trip_id'].isin(dirty_rows['trip_id'])]
    .sort_values(['cal_id', 'trip_id'])
)

In [108]:
dirty_df.shape

(39, 20)

In [114]:
wego_trim.loc[wego_trim['tsp_status'] == 'tsp_on_hdwy_cond'].shape

(509314, 20)

In [94]:
def extract_indexes(df):
    return_list = []
    for x in df:
        return_list.append(x.name())
    return return_list

In [ ]:
# Save indexes of dirty trips to list
dirty_indexes_1 = extract_indexes(wego_trim.loc[wego_trim['cal_id'].isin(dirty_rows['cal_id'])].loc[wego_trim['trip_id'].isin(dirty_rows['trip_id'])].sort_values(['cal_id', 'trip_id'])

In [ ]:
# Remove dirty trips
wego_trim.drop